# Esta es una adaptacion de una notebook encontrada buscando GPT2

Vamos a reentrenar una red neuronal de generacion de texto para adaptarlo a la forma de escritura de nuestros tweeteros gratis usando la GPU de Collaboratory y la libreria `gpt-2-simple`

Para saber más de `gpt-2-simple`, Visitar [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). Tambien pueden leer el post original que usamos de base para esta notebook: [blog post](https://minimaxir.com/2019/09/howto-gpt2/)
Link a la notebook original: https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce by [Max Woolf](http://minimaxir.com)



## Se le realizaron muchos cambios y adaptaciones.

**Al ejecutar en Collaboratory hace falta prestar atencion a la linea que se conecta con GDrive**


Ejecutar en Google Colab!

Si se quiere ejecutar de manera local se requiere Python 3.6 y tensorflow 1.15

Se puede conseguir corriendo:

    conda create --name GPT2GPU python=3.6 jupyter -y
    conda activate GPT2GPU
    conda install --name GPT2GPU tensorflow-gpu=1.15 --channel conda-forge -y

ó para correrlo con CPU

    conda create --name GPT2CPU python=3.6 jupyter -y
    conda activate GPT2CPU
    conda install --name GPT2CPU tensorflow=1.15 --channel conda-forge -y


## Configuracion antes de ejecutar

se pueden cambiar algunos valores aqui y volver a correr toda la notebook para que entrene y guarde otro modelo

In [1]:
# Archivo para el fine Tunning, tiene que estar en la raiz de tu drive o en ../datasets/xxx.txt
FILENAME = "FDT_to_JPC.txt"

# Nombre del Modelo que vas a generar despues del finetunning, Ejemplo: FDT ó JPC
MODEL_NAME = "FDT"   

# Nombre del Modelo que vas a usar de base de GPT2, en Colab usar solo: 133M ó 355M, en Anaconda se puede 774M
GTP2_MODEL_NAME = "355M"

# Ejecutar un modelo preentrenado o hacer finetunning:  EXECUTE o FINETUNNING
EXECUTE_OR_FINETUNNING = "EXECUTE"

# COLAB or LOCAL
ENVIRONMENT = "LOCAL" 



### NO TOCAR
MODEL_DIRS='models'
if ENVIRONMENT=="LOCAL":
    MODEL_DIRS ='../' + MODEL_DIRS
    FILENAME = "../datasets/" + FILENAME  
    CHECKPOINT_DIR=MODEL_DIRS
else:
    CHECKPOINT_DIR="checkpoint"

#  Empieza la notebook

In [2]:
# Si esto no dice que tenes la version 1.15 de tensorflow no va a funcionar!
!pip show tensorflow

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/santiago/anaconda3/envs/GPT2CPU/lib/python3.6/site-packages
Requires: termcolor, absl-py, wheel, numpy, grpcio, six, tensorflow-estimator, opt-einsum, keras-preprocessing, google-pasta, astor, wrapt, protobuf, keras-applications, tensorboard, gast
Required-by: 


In [3]:
if ENVIRONMENT=="COLAB":
    %tensorflow_version 1.x
    from google.colab import files
else:
    !pip install -q tensorflow-plot==0.3.0
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory usa tanto una Nvidia T4 GPU o una Nvidia K80 GPU. La T4 es ligeramente mas rapida que la K80

Podes verificar en que estas corriendo usando esta celda

In [4]:
!nvidia-smi

Thu Aug  6 14:42:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 2060    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P8     9W /  N/A |   1139MiB /  5934MiB |     36%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [5]:
import os
if not os.path.exists(MODEL_DIRS+"/"+GTP2_MODEL_NAME):
    gpt2.download_gpt2(model_name=GTP2_MODEL_NAME,model_dir=MODEL_DIRS)

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [6]:
if ENVIRONMENT=="COLAB":
    gpt2.mount_gdrive()

## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [7]:
#FDT = "FDT_to_JPC.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [8]:
if ENVIRONMENT=="COLAB":
    gpt2.copy_file_from_gdrive(FILENAME)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [9]:
if EXECUTE_OR_FINETUNNING != "EXECUTE":
    sess = gpt2.start_tf_sess()

    gpt2.finetune(sess,
              dataset=FILENAME,
              model_name=GTP2_MODEL_NAME,
              model_dir=MODEL_DIRS,
              steps=100,
              restore_from='fresh',
              run_name=MODEL_NAME,
              print_every=10,
              sample_every=10,
              save_every=10,
              checkpoint_dir=CHECKPOINT_DIR
              )
    if ENVIRONMENT=="COLAB":    
        gpt2.copy_checkpoint_to_gdrive(run_name=MODEL_NAME)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [10]:
if EXECUTE_OR_FINETUNNING == "EXECUTE":
    if ENVIRONMENT=="COLAB":
        gpt2.copy_checkpoint_from_gdrive(run_name=MODEL_NAME)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [11]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=MODEL_NAME,checkpoint_dir=MODEL_DIRS)

Loading checkpoint ../models/FDT/model-2000
INFO:tensorflow:Restoring parameters from ../models/FDT/model-2000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [13]:
gpt2.generate(sess, run_name=MODEL_NAME,truncate="<|endoftext|>",checkpoint_dir=MODEL_DIRS)

 que insensate y reivindicaron en la presente y future la bajeza de los clases afechas saliendo la concepcion #DondeEsta



If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [20]:
gpt2.generate(sess,
              checkpoint_dir=CHECKPOINT_DIR,
              run_name=MODEL_NAME,
              length=250,
              temperature=.9,
              prefix="[POST] A partir de hoy se reduce el impuesto a las ganancias a 0% \n[RESPONSE]",
              nsamples=5,
              batch_size=5,
              truncate="<|endoftext|>"
              )

[POST] A partir de hoy se reduce el impuesto a las ganancias a 0% 
[RESPONSE] @diegocabot Excelente ! No hay diga cuenta lo que dicen....

[POST] A partir de hoy se reduce el impuesto a las ganancias a 0% 
[RESPONSE] @martintetaz La sororidad de impuestos tuvo marginales dictamos que son lograr que lo denuncio hoy en 4 años de compañeros y doubidos. Le hablaba a un canjejar ▀ https://t.co/Vya6C5eH7a5 ▀ https://t.co/pEDAM5BxCC

[POST] A partir de hoy se reduce el impuesto a las ganancias a 0% 
[RESPONSE] @JuanGrabois Todo lo que hizo, uno de los days, lo que sucedió en los años oscuros de nuestro país. Eso pensé que su decisión dice Daniel Goleman. 
Fue la convocación deloridad entre todas las etapas. Encuentran la bajeza que era ministra. https://t.co/tcbEAUr83u

[POST] A partir de hoy se reduce el impuesto a las ganancias a 0% 
[RESPONSE] @lucasilardo @alferdez Algún día de lo que es un chiste? Me da asco!! Mano! Vamos a creer! 

[POST] A partir de hoy se reduce el impuesto a las gana

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      checkpoint_dir=CHECKPOINT_DIR,
                      run_name=MODEL_NAME,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
#!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.